# NLP training example
In this example, we'll train an NLP model for sentiment analysis of tweets using spaCy.

First we download spaCy language libraries.

In [1]:
!python -m spacy download xx_ent_wiki_sm

You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')


And import the boilerplate code.

In [2]:
from __future__ import unicode_literals, print_function

import boto3
import json
import numpy as np
import pandas as pd
import spacy

## Data prep

Download the dataset from S3

In [ ]:
S3_BUCKET = "verta-strata"
EN_S3_KEY = "english-tweets.csv"
EN_FILENAME = EN_S3_KEY
DE_S3_KEY = "german-tweets.csv"
DE_FILENAME = DE_S3_KEY

boto3.client('s3').download_file(S3_BUCKET, EN_S3_KEY, EN_FILENAME)
boto3.client('s3').download_file(S3_BUCKET, DE_S3_KEY, DE_FILENAME)

Clean and load data using our library.

In [ ]:
import utils

en_data = pd.read_csv(EN_FILENAME)
de_data = pd.read_csv(DE_FILENAME)

data = pd.concat([en_data, de_data], axis=0)
data = data.sample(frac=1).reset_index(drop=True)
utils.clean_data(data)

data.head()

## Set up ModelDB
ModelDB organizes our work, and enables us to log and version metadata.

In [ ]:
from verta import Client

client = Client('https://dev.verta.ai')
client.set_project('Tweet Classification')
client.set_experiment('SpaCy')
run = client.set_experiment_run()

We'll first record our code, configuration, dataset, and environment versions to a ModelDB repository.

In [ ]:
repo = client.set_repository('Verta Strata')
commit = repo.get_commit(branch='master')

In [ ]:
from verta.code import Notebook
from verta.configuration import Hyperparameters
from verta.dataset import S3
from verta.environment import Python

code_ver = Notebook()
config_ver = Hyperparameters({'n_iter': 20})
dataset_ver = S3([
    "s3://{}/{}".format(S3_BUCKET, EN_S3_KEY),
    "s3://{}/{}".format(S3_BUCKET, DE_S3_KEY),
])
env_ver = Python()

commit.update("notebooks/tweet-analysis", code_ver)
commit.update("config/hyperparams", config_ver)
commit.update("data/tweets", dataset_ver)
commit.update("env/python", env_ver)
commit.save("Deployment-ready sentiment analysis")

commit

## Train the model
We'll use a pre-trained model from spaCy and fine tune it in our new dataset.

In [ ]:
nlp = spacy.load('xx_ent_wiki_sm')

Update the model with the current data using our library.

In [ ]:
import training

training.train(nlp, data, n_iter=20)

## Save and version the model
We log the model itself as an artifact to ModelDB.

In [ ]:
run.log_model(nlp)

And finally, link the commit to our Experiment Run.

In [ ]:
run.log_commit(
    commit,
    {
        'notebook': "notebooks/tweet-analysis",
        'hyperparameters': "config/hyperparams",
        'training_data': "data/tweets",
        'python_env': "env/python",
    },
)

## Deployment

Great! Now you have a model that you can use to run predictions against. Follow the next step of this tutorial to see how to do it.